In [ ]:
from __future__ import annotations

import os
from pathlib import Path

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import mplhep as hep
import numpy as np
import pandas as pd
from scipy import interpolate
from tqdm import tqdm

plt.style.use(hep.style.CMS)
hep.style.use("CMS")
formatter = mticker.ScalarFormatter(useMathText=True)
formatter.set_powerlimits((-3, 3))
plt.rcParams.update({"font.size": 20})

In [ ]:
def mxmy(sample):
    mY = int(sample.split("-")[-1])
    mX = int(sample.split("NMSSM_XToYHTo2W2BTo4Q2B_MX-")[1].split("_")[0])

    return (mX, mY)

In [ ]:
plot_dir = Path("../../../../plots/XHY/Limits/25Feb13")
plot_dir.mkdir(parents=True, exist_ok=True)

In [ ]:
cards_dir = Path("/eos/uscms/store/user/rkansal/bbVV/cards/25Feb12ResFixes")
# temporary backup directory while jobs are still running
backup_dir = Path("/eos/uscms/store/user/rkansal/bbVV/cards/25Feb12Res_31")
samples = [d.name for d in cards_dir.iterdir() if d.is_dir()]

Load / process limits

In [ ]:
def _parse_limits(lines, limits):
    nums = 0
    for i in np.arange(len(lines) - 1, -1, -1):
        line = lines[i][:-1]
        for key in limits:
            start_str = f"Expected {key}%: r < "
            if line.startswith(start_str):
                limits[key].append([mx, my, float(line.split(start_str)[1])])
                nums += 1

        if nums == 5:
            break


limits = {" 2.5": [], "16.0": [], "50.0": [], "84.0": [], "97.5": []}

for sample in tqdm(samples):
    limits_path = Path(f"{cards_dir}/{sample}/AsymptoticLimits.txt")
    backup_path = Path(f"{backup_dir}/{sample}/AsymptoticLimits.txt")
    mx, my = mxmy(sample)
    if limits_path.exists():
        with limits_path.open() as f:
            lines = f.readlines()
    elif backup_path.exists():
        print(f"Using backup path for {sample}")
        with backup_path.open() as f:
            lines = f.readlines()
    else:
        print(f"Missing {sample}")
        continue

    _parse_limits(lines, limits)

for key in limits:
    limits[key] = np.array(limits[key])

In [ ]:
limit_dir = plot_dir / "limits"
limit_dir.mkdir(exist_ok=True)

for key, limit in limits.items():
    df = pd.DataFrame(limit, columns=["MX", "MY", "Limit (fb)"])
    df.to_csv(f"{limit_dir}/limits_{key}.csv")

Plot

In [ ]:
def scatter2d(arr, title, name):
    fig, ax = plt.subplots(figsize=(14, 12))
    mappable = plt.scatter(
        arr[:, 0],
        arr[:, 1],
        s=150,
        c=arr[:, 2],
        cmap="turbo",
        norm=matplotlib.colors.LogNorm(vmin=0.01, vmax=100),
    )
    plt.title(title)
    plt.xlabel(r"$m_X$ (GeV)")
    plt.ylabel(r"$m_Y$ (GeV)")
    plt.colorbar(mappable)
    plt.savefig(name, bbox_inches="tight")

In [ ]:
def scatter2d_overlay(arr, overlay_arr, title, name):
    fig, ax = plt.subplots(figsize=(14, 12))
    mappable = ax.scatter(
        arr[:, 0],
        arr[:, 1],
        s=150,
        c=arr[:, 2],
        cmap="turbo",
        norm=matplotlib.colors.LogNorm(vmin=0.01, vmax=100),
    )
    _ = ax.scatter(
        overlay_arr[:, 0],
        overlay_arr[:, 1],
        s=150,
        marker="s",
        alpha=0.5,
        c=np.ones(overlay_arr.shape[0]),
        vmax=1,
    )
    plt.title(title)
    plt.xlabel(r"$m_X$ (GeV)")
    plt.ylabel(r"$m_Y$ (GeV)")
    plt.colorbar(mappable)
    plt.savefig(name, bbox_inches="tight")

In [ ]:
def colormesh(xx, yy, lims, label, name, figsize=(12, 8)):
    fig, ax = plt.subplots(figsize=figsize)
    _ = plt.pcolormesh(
        xx, yy, lims, norm=matplotlib.colors.LogNorm(vmin=0.05, vmax=1e4), cmap="turbo"
    )
    # plt.title(title)
    plt.xlabel(r"$m_X$ (GeV)")
    plt.ylabel(r"$m_Y$ (GeV)")
    plt.colorbar(label=label)
    hep.cms.label("Work in Progress", data=True, lumi="138", ax=ax)
    plt.savefig(name, bbox_inches="tight")

In [ ]:
np.min(limits["50.0"][:, 2])

In [ ]:
with Path("amitav_limits.csv").open() as f:
    alimits = pd.read_csv(f)

In [ ]:
sb_better = []

for mx, my, lim in limits["50.0"]:
    match = (alimits["MX"] == mx) * (alimits["MY"] == my)
    if np.any(match):
        alim = alimits["Limit (fb)"][match].values[0]

    if alim < lim:
        pbetter = (lim - alim) / lim * 100
        print(f"Semiboosted better for ({mx}, {my}) by {pbetter:.2f}%")
        sb_better.append([mx, my])

sb_better = np.array(sb_better)

In [ ]:
mymax = 250
mxs = np.logspace(np.log10(600), np.log10(3999), 100, base=10)
mys = np.logspace(np.log10(60), np.log10(mymax), 100, base=10)

xx, yy = np.meshgrid(mxs, mys)

interpolated = {}
grids = {}

for key, val in limits.items():
    interpolated[key] = interpolate.LinearNDInterpolator(val[:, :2], np.log(val[:, 2]))
    grids[key] = np.exp(interpolated[key](xx, yy))

In [ ]:
for key, grid in grids.items():
    label = (
        f"{key}% expected exclusion limits (fb)"
        if key != "50.0"
        else "Median expected exclusion limits (fb)"
    )
    colormesh(xx, yy, grid, label, f"{plot_dir}/upper{mymax}_mesh_{key}_turbo.pdf")

In [ ]:
key = "50.0"
val = limits[key]
scatter2d(val, f"Median expected exclusion limits (fb)", f"{plot_dir}/scatter_{key}.pdf")

In [ ]:
scatter2d_overlay(
    limits[key],
    sb_better,
    f"Median expected exclusion limits (fb)",
    f"{plot_dir}/scatter_overlay.pdf",
)

In [ ]:
combined_limits = []


for mx, my, alim in alimits.values[:, 1:]:
    match = (limits["50.0"][:, 0] == mx) * (limits["50.0"][:, 1] == my)
    if np.any(match):
        blim = limits["50.0"][:, 2][match].squeeze()
        lim = np.minimum(alim, blim)
    else:
        lim = alim

    combined_limits.append([mx, my, lim])


combined_limits = np.array(combined_limits)

In [ ]:
mxs = np.logspace(np.log10(600), np.log10(3999), 300, base=10)
mys = np.logspace(np.log10(60), np.log10(2800), 300, base=10)
cxx, cyy = np.meshgrid(mxs, mys)

val = combined_limits
combined_interp = interpolate.LinearNDInterpolator(val[:, :2], np.log(val[:, 2]))
combined_grid = np.exp(combined_interp(cxx, cyy))

In [ ]:
colormesh(
    cxx,
    cyy,
    combined_grid,
    "Median expected exclusion limits (fb)",
    f"{plot_dir}/combined_mesh_{key}_turbo.pdf",
    figsize=(12, 8),
)